In [ ]:
import torchao
from torchao.dtypes import NF4Tensor
from torchao.dtypes.nf4tensor import to_nf4
import torch

In [ ]:
# Must show all graph breaks are not seen with torch.compile
import os
os.environ["TORCHDYNAMO_VERBOSE"] = "1"
os.environ["TORCHINDUCTOR_FORCE_DISABLE_CACHES"] = "1"
os.environ["TORCHINDUCTOR_COMPILE_THREADS"] = "1"

import logging
torch._inductor.config.debug = True
torch._logging.set_logs(
    dynamo = logging.WARN,
    inductor = logging.WARN,
    graph_breaks = True,
    recompiles = True,
    recompiles_verbose = True,
    compiled_autograd_verbose = True,
    # aot_joint_graph = True, # Enable for more logs
    # aot_graphs = True,
)
torch._dynamo.config.verbose = True
torch._dynamo.config.suppress_errors = False

In [ ]:
import torch
x = torch.randn(16, 16)  # Example full precision tensor
x_nf4 = to_nf4(x, 16, 1)

In [ ]:
x - x_nf4.get_original_weight()

In [ ]:
import torch
torch_compile_options = torch_compile_options = {
    "epilogue_fusion"   : True,
    "max_autotune"      : True,
    "shape_padding"     : True,
    "trace.enabled"     : True,
    "triton.cudagraphs" : False,
}

# lets compile a basic example
@torch.compile(fullgraph = True)
def compiled_get_original(x):
  return x.get_original_weight()

In [ ]:
compiled_get_original(x_nf4)